In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from subprocess import call
import os
import pandas as pd
import random
import glob
import sys
sys.path.append('../')
from package_global_functions import *

extSSDpath = getExternalSSDpath()
if os.path.exists(extSSDpath):
    resPath = extSSDpath + getProjectFoldername() + '/gillespie_sim_ci/results'
else:
    resPath = '/results'

In [22]:
# get stationary time from simulations

def search_time(w,t,evo,sig=0):
    """
    sig=0 -> no gaussian filter; sig != 0 -> apply gaussian filter with this sigma
    """
    if sig:
        evo_mod = gaussian_filter1d(evo, sig)
    else:
        evo_mod = evo
    time = float('nan')
    for i in range(len(evo_mod)-w):
        block_avg = np.average(evo_mod[i:i+w])
        if abs(evo_mod[i+w+1] - block_avg) < t:
            time = i+w+1
            break
    return time

# from gillespie simulations
def get_tss(w,tol):
    global pis, qs, l, lci, N, maxTime, Nrea, ic, ci_kwargs
    pichain = ','.join([str(pi) for pi in pis])
    qchain = ','.join([str(q) for q in qs])
    ci_kwargs_chain = ','.join([str(cikw) for cikw in ci_kwargs])
    simCall = f'python LES_model_gill.py -pis {pichain} -qs {qchain} -l {l} -lci {lci} -ci_kwargs {ci_kwargs_chain} '
    simCall += f'-N {N} -maxTime {maxTime} -Nrea {Nrea} -ic {ic} --time_evo'
    call(simCall, shell=True)
    call('tar -xzf sim_results_evos.tar.gz', shell=True)
    evoFiles = glob.glob('sim_results_evos/*')
    all_tss = []
    for testid,f in enumerate(evoFiles):
        df = pd.read_csv(f)
        tss_rea = []
        for k in range(0,len(pis)+1):
            tss_fi = search_time(w,tol,df[f'f{k}'])
            tss_fi = float(df['time'].iloc[tss_fi])
            tss_rea.append(tss_fi)
        all_tss.append(max(tss_rea))
        # fig, ax = plt.subplots(1,1, figsize=(4,4), constrained_layout=True)
        # ax.set_xscale('symlog')
        # iters = list(range(len(df)))
        # ax.plot(iters, df['f0'], color='r')
        # ax.plot(iters, df['f1'], color='g')
        # ax.plot(iters, df['f2'], color='b')
        # ax.axvline(max(tss_rea), 0, 1)
        # fig.savefig(f'test_tss_{testid}.png')
    return all_tss


# from agent based model simulations
# remain that in fortran code I use the convention of ci modes 1,2,3 for linear,sig1,sig2 respectively!!!
# for this reason cimode=cimode+1 below;)
def get_tss_AB(w,tol):
    global pis, qs, l, lci, N, maxTime, Nrea, ic, ci_kwargs
    froute = '../clean_version_ci/'
    fin_file = 'input_template.txt'
    fex_file = 'main.x'
    pichain = ','.join([str(pi) for pi in pis])
    qchain = ','.join([str(q) for q in qs])
    ci_kwargs_chain = ','.join([str(cikw) for cikw in ci_kwargs])
    change_sim_input(froute, fin_file, pis=pis, qs=qs, lamb=l, max_time=maxTime, N_sites=len(pis), N_bots=N, bots_per_site=[N, 0, 0], ic='N', 
                     lci=lci, cimode=ci_kwargs[0]+1)
    if len(ci_kwargs) > 1:
        change_sim_input(ci_x0=ci_kwargs[1], ci_a=ci_kwargs[2])
    wd = os.getcwd()
    # Execute simulations:
    os.chdir(froute)
    call("./"+fex_file+f" {random.randint(0,100000000)} {Nrea}", shell=True)
    os.chdir(wd)
    # Save the time evolutions:
    call(f'tar -xzf {froute}time_evo_csv.tar.gz', shell=True)
    # call(f'mv time_evo_csv {newFolderName}', shell=True)
    # call(f'mkdir -p {path}', shell=True)
    # if os.path.exists(f'{path}/{newFolderName}'):
    #     call(f'rm -r {path}/{newFolderName}', shell=True)
    # call(f'mv {newFolderName} {path}', shell=True)
    evoFiles = glob.glob('time_evo_csv/*')
    all_tss = []
    for testid,f in enumerate(evoFiles):
        df = pd.read_csv(f)
        tss_rea = []
        for i in range(0,len(pis)+1):
            tss_fi = search_time(w,tol,df[f'f{i}'])
            tss_rea.append(tss_fi)
        all_tss.append(max(tss_rea))
        # plots for testing:
        # fig, ax = plt.subplots(1,1, figsize=(4,4), constrained_layout=True)
        # ax.set_xscale('symlog')
        # iters = list(range(len(df)))
        # ax.plot(iters, df['f0'], color='r')
        # ax.plot(iters, df['f1'], color='g')
        # ax.plot(iters, df['f2'], color='b')
        # ax.axvline(max(tss_rea), 0, 1)
        # fig.savefig(f'test_tss_{testid}.png')      
    return all_tss

# Using Gillespie Simulations

In [ ]:
pis, qs = [0.1, 0.1], [9.0, 10.0]
l, lci, N, maxTime, Nrea, ic = 0.6, 1.0, 500, 90.0, 5, 'N'
ci_kwargs = [2, 0.3, 10.0]
w, tol = 1000, 1e-4
get_tss(w, tol)

In [23]:
maxTime = 90.0
Nrea = 10
w, tol = 1000, 1e-4

ci_kwargs = [2, 0.3, 10.0]
times = get_tss(w, tol)
print(np.average(times))

ci_kwargs = [0, ]
times = get_tss(w, tol)
print(np.average(times))


31.418876790525342
31.28293218657363


# Using agent based model

In [ ]:
pis, qs = [0.1, 0.1], [9.0, 10.0]
l, lci, N, maxTime, Nrea, ic = 0.6, 1.0, 500, 3000, 5, 'N'
ci_kwargs = [2, 0.3, 10.0]
w, tol = 50, 1e-4
get_tss_AB(w, tol)

In [16]:
w, tol = 50, 1e-4

In [18]:
maxTime = 2000
Nrea = 10
ci_kwargs = [2, 0.3, 10.0]
times = get_tss_AB(w, tol)
print(np.average(times))

359.9


In [21]:
ci_kwargs = [0, ]
times = get_tss_AB(w, tol)
print(np.average(times))

428.4
